In [65]:


# from langchain.llms import LM
# chat = ChatOpenAI(temperature=0.0, base_url="http://192.168.12.210:1235/v1", api_key="not-needed")


from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    base_url="http://192.168.12.210:1234/v1",
    api_key="not-needed",
    model="meta-llama-3.1-8b-instruct", # "qwen2.5-coder-3b-instruct",
    temperature=0.1,
    )

llm2 = ChatOpenAI(
    base_url="http://192.168.12.210:1234/v1",
    api_key="not-needed",
    model="qwen2.5-coder-3b-instruct",
    temperature=0.0,
    )



In [ ]:
llm.invoke("What is your favorite color?")

In [ ]:


template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = llm.invoke(customer_messages)

In [ ]:
print(customer_response.content)


In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = llm.invoke(service_messages)
print(service_response.content)

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [63]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just before i needed it. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

# I think my wife liked it so much she was speechless. \


review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer 'true' if yes, 'false' if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [67]:

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template="For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer 'true' if yes, 'false' if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n"), additional_kwargs={})]


In [68]:
messages = prompt_template.format_messages(text=customer_review)
response = llm.invoke(messages, max_tokens=60)
print(response.content)

```python
import json

text = """
This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. 
It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. 
So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. 
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.
"""

def extract_info(text):
    gift = 'true' if 'gift' in text.lower() else 'false'
    delivery_days = -1
    sentences = text.split('. ')
    price_value = []
    
    for sentence in sentences:
        if 'days' in sentence:
            delivery_days = int(sentence.split('in ')[1].split(' days')[0])
        elif 'expensive' in sentence or 'worth it' in sentence:
            price_value.append(sentence)
            
    return json.dumps({
        "gift": gift,
        "delivery_day

In [69]:
type(response.content)

str

In [70]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [71]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [79]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift? \
                             Answer 'true' if yes,\
                             'false' if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive as float? If this \
                                      information is not found,\
                                      output -1.0 .")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [80]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [81]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift?                              Answer 'true' if yes,                             'false' if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive as float? If this                                       information is not found,                                      output -1.0 .
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [85]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift? \
Answer 'true' if yes, 'false' if not or unknown.

delivery_days: How many days did it take for the product\
to arrive as float? If this information is not found, output -1.0 .

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [86]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift? Answer 'true' if yes, 'false' if not or unknown.

delivery_days: How many days did it take for the productto arrive as float? If this information is not found, output -1.0 .

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```"

In [87]:
response = llm.invoke(messages, max_tokens=60)
print(response.content)


```json
{
    "gift": "true",
    "delivery_days": 2.0,
    "price_value": "It's slightly more expensive than the other leaf blowers out there"
}
```

Here is how I extracted this information:

- `gift`: The text mentions


In [78]:
output_dict = output_parser.parse(response.content)
print(output_dict)
type(output_dict)
output_dict.get('delivery_days')

{'gift': False, 'delivery_days': 2.0, 'price_value': "It's slightly more expensive than the other leaf blowers out there"}


2.0